# Movie Sentiment Analysis with Keras

In [1]:
# uncomment these for Google collab, will have already been installed in local environment 
# if 'pip install -r requirements.txt' has been run
#!pip install nltk
#!pip install --upgrade gensim

import numpy as np
import os
import os.path

from pdb import set_trace
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
import nltk


import glob
from gensim.models import Word2Vec

import time

[nltk_data] Downloading package punkt to /Users/swami/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# MacOSX: See https://www.mkyong.com/mac/wget-on-mac-os-x/ for wget
if not os.path.isdir('../aclImdb'):
    if not os.path.isfile('../aclImdb_v1.tar.gz'):
      !wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz 

    if not os.path.isdir('../aclImdb'):  
      !tar -xf aclImdb_v1.tar.gz 

In [3]:
time_beginning_of_notebook = time.time()
SAMPLE_SIZE=3000
positive_sample_file_list = glob.glob(os.path.join('../aclImdb/train/pos', "*.txt"))
positive_sample_file_list = positive_sample_file_list[:SAMPLE_SIZE]

negative_sample_file_list = glob.glob(os.path.join('../aclImdb/train/neg', "*.txt"))
negative_sample_file_list = negative_sample_file_list[:SAMPLE_SIZE]

import re

# load doc into memory
# regex to clean markup elements 
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r', encoding='utf8')
    # read all text
    text = re.sub('<[^>]*>', ' ', file.read())
    #text = file.read()
    # close the file
    file.close()
    return text


In [4]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

df_positives = pd.DataFrame({'reviews':[load_doc(x) for x in positive_sample_file_list], 'sentiment': np.ones(SAMPLE_SIZE)})
df_negatives = pd.DataFrame({'reviews':[load_doc(x) for x in negative_sample_file_list], 'sentiment': np.zeros(SAMPLE_SIZE)})

print("Positive review(s):", df_positives['reviews'][1][:100])
print("Negative review(s):", df_negatives['reviews'][1][:100])

df = pd.concat([df_positives, df_negatives], ignore_index=True)

df = shuffle(df)

X_train, X_test, y_train, y_test = train_test_split(df['reviews'], df['sentiment'], test_size=0.25)


Positive review(s): Bizarre horror movie filled with famous faces but stolen by Cristina Raines (later of TV's "Flamingo
Negative review(s): Well...tremors I, the original started off in 1990 and i found the movie quite enjoyable to watch. h


### Logic to compute DAN model

In [171]:
#ML STUDY GROUP
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

class PreProcessor:
    def __init__(self,REVIEWS,REVIEWS_VAL,LABELS,LABELS_VAL,WE_FILE):
        self.reviews = REVIEWS
        self.reviews_val = REVIEWS_VAL
        self.labels = LABELS
        self.labels_val = LABELS_VAL
        self.we_file = WE_FILE

    def tokenize(self):
#         set_trace()
        print(self.reviews[0])

        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(self.reviews)

        self.sequences = tokenizer.texts_to_sequences(self.reviews)
        self.sequences_val = tokenizer.texts_to_sequences(self.reviews_val)

        self.word_index = tokenizer.word_index
        print("Found %s unique tokens" %(len(self.word_index)))

    def make_data(self):
        self.MAX_SEQUENCE_LENGTH = max([len(self.sequences[i]) for i in range(len(self.sequences))])
        print("self.MAX_SEQUENCE_LENGTH: {}".format(self.MAX_SEQUENCE_LENGTH))

        review = pad_sequences(self.sequences,maxlen=self.MAX_SEQUENCE_LENGTH)
        review_val = pad_sequences(self.sequences_val,maxlen=self.MAX_SEQUENCE_LENGTH)
        
        labels = to_categorical(self.labels)
        labels_val = to_categorical(self.labels_val)

        print("Shape of data tensor: " +str(review.shape))
        print("Shape of label tensor: " +str(labels.shape))

        return review, review_val, labels, labels_val
        
    def get_word_embedding_matrix(self,EMBEDDING_DIM=100):
        embeddings_index = {}

        if self.we_file == "rand":
            return None

        f = open(self.we_file)

        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        f.close()

        print('Found %s word vectors.' % len(embeddings_index))

        self.embedding_matrix = np.zeros((len(self.word_index)+1, EMBEDDING_DIM))

        for word, i in self.word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                self.embedding_matrix[i] = embedding_vector

        return self.embedding_matrix


In [172]:
import argparse
import numpy as np

from dan.custom_layers import AverageWords, WordDropout

from keras.layers import Embedding, Dense, Input, BatchNormalization, Activation, Dropout
from keras.models import Sequential
from keras.optimizers import Adagrad, Adam
from keras import backend as K

from pdb import set_trace

embedding_dim = 300
num_hidden_layers = 3
num_hidden_units = 300
num_epochs = 100
batch_size = 512
dropout_rate = 0.2
word_dropout_rate = 0.3
activation = 'relu'

args = {}
args['We']='data/glove.6B.300d.txt'
args['Wels']='' ### rand or ''
args['model']='dan'  ### nbow OR dan
args['wd']='y'

reviews=X_train.values
reviews_val=X_test.values
labels=y_train.values
labels_val=y_test.values

In [173]:
# reviews_val.values

In [174]:
pp = PreProcessor(reviews,reviews_val,labels,labels_val,args['We'])
pp.tokenize()

reviews,reviews_val,labels,labels_val = pp.make_data()

embedding_matrix = pp.get_word_embedding_matrix(embedding_dim)


This movie didn't really surprise me, as such, it just got better and better. I thought: "Paul Rieser wrote this, huh? Well...we'll see how he does..." Then I saw Peter Falk was in it. I appreciate Colombo. Even though I was never a big fan of the show, I've always liked watching Peter Falk.   The performances of Peter and Paul were so natural that I felt like a fly on the wall. They played off of each other so well that I practically felt giddy with enjoyment! ...And I hadn't even been drinking!  This movie was so well done that I wanted to get right on the phone to Paul and let him know how much I enjoyed it! but I couldn't find his number. Must be unlisted or something.  This was one of those movies that I had no idea what it was going to be about or who was in it or anything. It just came on and I thought:"Eh, why not? Let's see. If I don't like it - I don't have to watch it..." ...and I ended up just loving it!
Found 41247 unique tokens
self.MAX_SEQUENCE_LENGTH: 2473
Shape of data

In [175]:
model = Sequential()

if args['Wels'] == "rand":
    model.add(Embedding(len(pp.word_index) + 1,embedding_dim,input_length=pp.MAX_SEQUENCE_LENGTH,trainable=False))
else:
    model.add(Embedding(len(pp.word_index)+1,embedding_dim,weights=[embedding_matrix],input_length=pp.MAX_SEQUENCE_LENGTH,trainable=False))

if args['wd'] == 'y':
    model.add(WordDropout(word_dropout_rate))
model.add(AverageWords())


In [176]:
print('reviews.shape: ' + str(reviews.shape))
print('reviews_val.shape: ' + str(reviews_val.shape))
print('labels.shape: ' + str(labels.shape))
print('labels_val.shape: ' + str(labels_val.shape))
labels.shape[0]

reviews.shape: (4500, 2473)
reviews_val.shape: (1500, 2473)
labels.shape: (4500, 2)
labels_val.shape: (1500, 2)


4500

In [177]:
if args['model'] == 'dan':
    for i in range(num_hidden_layers):
        model.add(Dense(num_hidden_units))
        model.add(BatchNormalization())
        model.add(Activation(activation))
        model.add(Dropout(dropout_rate))

model.add(Dense(labels.shape[1]))
model.add(BatchNormalization())
model.add(Dropout(dropout_rate))
model.add(Activation('softmax'))

adam = Adam()
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy','categorical_accuracy'])

model.summary()

model.fit(reviews,labels,batch_size=batch_size,epochs=num_epochs,\
          validation_data=(reviews_val,labels_val))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 2473, 300)         12374400  
_________________________________________________________________
word_dropout_13 (WordDropout (None, 2473, 300)         0         
_________________________________________________________________
average_words_13 (AverageWor (None, 300)               0         
_________________________________________________________________
dense_165 (Dense)            (None, 300)               90300     
_________________________________________________________________
batch_normalization_165 (Bat (None, 300)               1200      
_________________________________________________________________
activation_165 (Activation)  (None, 300)               0         
_________________________________________________________________
dropout_165 (Dropout)        (None, 300)               0         
__________